In [1]:
import logging 
from gensim import corpora, models, similarities, matutils
import numpy as np
from scipy import stats
from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
#from sklearn.semi_supervised import LabelSpreading
#from sklearn.neural_network import BernoulliRBM
#from sklearn.ensemble import RandomTreesEmbedding
from sklearn.pipeline import Pipeline
from math import log
from nltk.corpus import stopwords

import matplotlib.pyplot as plt

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
bigram=models.Phrases([line.lower().split() for line in open('attribute-Edward Poot.txt')])
#print(bigram['Verpleegkundige met ervaring in de palliatieve zorg'.lower().split()])

In [3]:
print(bigram['In staat zijn om er voor te zorgen dat het zelfmanagement van de client wordt bevorderd'.lower().split()])

[u'in_staat', u'zijn_om', u'er', u'voor', u'te', u'zorgen', u'dat', u'het', u'zelfmanagement', u'van', u'de_client', u'wordt', u'bevorderd']


In [97]:
stoplist1 = set('thuiszorg verpleegkundige voor vacature u je aan clienten de deze een en het in op van zorg met zijn bij 5 wij of te \
                werken is onze over als die om bezit niveau_4 functie bent binnen'.split())
stoplist2=set(stopwords.words('dutch'))

In [115]:
#stoplist=stoplist1.union(stoplist2)
#'het' in stoplist
#stoplist=stoplist2
set(["urbanplanners.txt","python"])

{'python', 'urbanplanners.txt'}

In [4]:
dictionary=corpora.Dictionary(bigram[line.lower().split()] for line in open('attribute-Edward Poot.txt'))
stoplist=stopwords.words('dutch')

stop_ids =[dictionary.token2id[stopword] for stopword in stoplist
          if stopword in dictionary.token2id]
#largeidf_ids=[tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if log(173820/docfreq,2)<6]
#print(largeidf_ids)
#once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.iteritems() if docfreq==1]
dictionary.filter_tokens(stop_ids)
dictionary.compactify()
print(dictionary)

In [5]:
dictionary.save('./clusterEdward/edwarddict.dict')

In [6]:
dictionary = corpora.Dictionary.load('./clusterEdward/edwarddict.dict')

In [7]:
print(dictionary)

Dictionary(1779 unique tokens: [u'vereisen', u'gastvrouw', u'verklaring_omtrent', u'digitaal', u'verwachten']...)


In [8]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('attribute-Edward Poot.txt'):
            yield dictionary.doc2bow(bigram[line.lower().split()])

In [9]:
corpus_memory_friendly=MyCorpus()
corpora.MmCorpus.serialize('./clusterEdward/edwardcorpus.mm', corpus_memory_friendly)

In [10]:
corpus=corpora.MmCorpus("./clusterEdward/edwardcorpus.mm")

In [11]:
nonempty_corpus=[doc for doc in corpus if doc != []]

In [12]:
empty_corpus_ids=[]
for i in range(len(corpus)):
    if not corpus[i]:
        empty_corpus_ids.append(i)
    else:
        continue

In [13]:
len(nonempty_corpus)

1205

In [28]:
#log_ent =models.LogEntropyModel(corpus)
#corpus_logent=log_ent[corpus]
tfidf=models.TfidfModel(nonempty_corpus)
corpus_tfidf = tfidf[nonempty_corpus]
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=100)
corpus_lsi=lsi[corpus_tfidf]
#print(corpus_logent)
#print(corpus_lsi)

In [29]:
lsi.save('./clusterEdward/edwardlsi.lsi')

In [30]:
tfidf.save('./clusterEdward/edwardtfidf.tfidf')

In [31]:
mycorpus_matrix = matutils.corpus2dense(corpus_lsi, num_terms=100).transpose()
#mycorpus_matrix = matutils.corpus2dense(corpus_tfidf, num_terms=19722).transpose()
#mycorpus_matrix = matutils.corpus2csc(corpus_lsi)

In [39]:
model= AgglomerativeClustering(n_clusters=3,linkage="ward",affinity="euclidean")
clustermembership=model.fit_predict(mycorpus_matrix)